In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import os

In [ ]:
# Load data and model (if model exists, otherwise need to train first)
# Note: This cell assumes model has been trained. If not, run implementation.ipynb first

# Check if model exists
if os.path.exists("tesla_lstm_model.h5"):
    model = load_model("tesla_lstm_model.h5")
    
    # Load and preprocess data
    df = pd.read_csv("dataset.csv")
    df = df.sort_values("date")
    df = df.dropna()
    
    features = df.drop(["date", "next_day_close"], axis=1)
    target = df["next_day_close"]
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = scaler.fit_transform(features)
    scaled_target = scaler.fit_transform(target.values.reshape(-1, 1))
    
    # Create sequences
    def create_sequences(X, y, time_steps=60):
        Xs, ys = [], []
        for i in range(time_steps, len(X)):
            Xs.append(X[i-time_steps:i])
            ys.append(y[i])
        return np.array(Xs), np.array(ys)
    
    time_steps = 60
    X, y = create_sequences(scaled_features, scaled_target, time_steps)
    
    # Split data
    split = int(0.8 * len(X))
    X_test = X[split:]
    y_test = y[split:]
    
    # Make predictions
    predictions = model.predict(X_test, verbose=0)
    predictions = scaler.inverse_transform(predictions)
    y_test_actual = scaler.inverse_transform(y_test)
    
    print("Model loaded successfully!")
    print(f"Test set size: {len(y_test_actual)} samples")
else:
    print("Model not found. Please run implementation.ipynb first to train the model.")
    print("Or ensure you have run the implementation notebook and have the variables available.")

In [ ]:
# Visualizations
if 'predictions' in locals() and 'y_test_actual' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Actual vs Predicted
    axes[0, 0].plot(y_test_actual, label='Actual', alpha=0.7, linewidth=1.5)
    axes[0, 0].plot(predictions, label='Predicted', alpha=0.7, linewidth=1.5)
    axes[0, 0].set_title('Actual vs Predicted Stock Prices', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Time Steps')
    axes[0, 0].set_ylabel('Price ($)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Prediction Errors
    errors = y_test_actual - predictions
    axes[0, 1].plot(errors, color='red', alpha=0.7, linewidth=1)
    axes[0, 1].axhline(y=0, color='black', linestyle='--', linewidth=1)
    axes[0, 1].set_title('Prediction Errors Over Time', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Time Steps')
    axes[0, 1].set_ylabel('Error ($)')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Scatter Plot - Actual vs Predicted
    axes[1, 0].scatter(y_test_actual, predictions, alpha=0.5, s=20)
    min_val = min(y_test_actual.min(), predictions.min())
    max_val = max(y_test_actual.max(), predictions.max())
    axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    axes[1, 0].set_title('Actual vs Predicted (Scatter)', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Actual Price ($)')
    axes[1, 0].set_ylabel('Predicted Price ($)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Error Distribution
    axes[1, 1].hist(errors, bins=50, edgecolor='black', alpha=0.7)
    axes[1, 1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
    axes[1, 1].set_title('Error Distribution', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Error ($)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('model_analytics.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\nAnalytics visualization saved as 'model_analytics.png'")
else:
    print("Cannot generate visualizations. Please run implementation.ipynb first.")


In [ ]:
# Detailed Statistics Table
if 'predictions' in locals() and 'y_test_actual' in locals():
    errors = y_test_actual - predictions
    abs_errors = np.abs(errors)
    percentage_errors = np.abs((y_test_actual - predictions) / y_test_actual) * 100
    
    stats_df = pd.DataFrame({
        'Metric': [
            'Mean Error',
            'Median Error',
            'Std Deviation of Errors',
            'Mean Absolute Error',
            'Median Absolute Error',
            'Max Absolute Error',
            'Min Absolute Error',
            'Mean Percentage Error',
            'Median Percentage Error',
            'Max Percentage Error'
        ],
        'Value': [
            f'${np.mean(errors):.2f}',
            f'${np.median(errors):.2f}',
            f'${np.std(errors):.2f}',
            f'${np.mean(abs_errors):.2f}',
            f'${np.median(abs_errors):.2f}',
            f'${np.max(abs_errors):.2f}',
            f'${np.min(abs_errors):.2f}',
            f'{np.mean(percentage_errors):.2f}%',
            f'{np.median(percentage_errors):.2f}%',
            f'{np.max(percentage_errors):.2f}%'
        ]
    })
    
    print("\n" + "="*70)
    print("DETAILED STATISTICS")
    print("="*70)
    print(stats_df.to_string(index=False))
    print("\n" + "="*70)
    
    # Performance Summary
    print("\nPERFORMANCE SUMMARY")
    print("-" * 70)
    if r2 > 0.7:
        print("   Model Performance: EXCELLENT (R² > 0.7)")
    elif r2 > 0.5:
        print("   Model Performance: GOOD (R² > 0.5)")
    elif r2 > 0.3:
        print("   Model Performance: MODERATE (R² > 0.3)")
    else:
        print("   Model Performance: POOR (R² < 0.3)")
    
    if mape < 10:
        print("   Prediction Accuracy: EXCELLENT (MAPE < 10%)")
    elif mape < 15:
        print("   Prediction Accuracy: GOOD (MAPE < 15%)")
    elif mape < 20:
        print("   Prediction Accuracy: MODERATE (MAPE < 20%)")
    else:
        print("   Prediction Accuracy: NEEDS IMPROVEMENT (MAPE > 20%)")
    
    if directional_accuracy > 55:
        print("   Directional Accuracy: GOOD (> 55%)")
    elif directional_accuracy > 50:
        print("   Directional Accuracy: MODERATE (> 50%)")
    else:
        print("   Directional Accuracy: POOR (< 50%)")
    
    print("\n" + "="*70)


In [ ]:
# Calculate comprehensive metrics
if 'predictions' in locals() and 'y_test_actual' in locals():
    # Basic metrics
    rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
    mae = mean_absolute_error(y_test_actual, predictions)
    r2 = r2_score(y_test_actual, predictions)
    
    # Additional metrics
    mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100
    percentage_accuracy = max(0, 100 - mape)
    
    # Directional accuracy
    actual_direction = np.diff(y_test_actual.flatten())
    predicted_direction = np.diff(predictions.flatten())
    directional_accuracy = np.mean((actual_direction * predicted_direction) > 0) * 100
    
    # Mean percentage error (bias)
    mean_percentage_error = np.mean((y_test_actual - predictions) / y_test_actual) * 100
    
    # Additional statistics
    avg_price = np.mean(y_test_actual)
    max_error = np.max(np.abs(y_test_actual - predictions))
    min_error = np.min(np.abs(y_test_actual - predictions))
    std_error = np.std(y_test_actual - predictions)
    
    print("="*70)
    print("COMPREHENSIVE MODEL ANALYTICS")
    print("="*70)
    print("\n1. ERROR METRICS")
    print("-" * 70)
    print(f"   Root Mean Squared Error (RMSE):     ${rmse:.2f}")
    print(f"   Mean Absolute Error (MAE):         ${mae:.2f}")
    print(f"   Mean Absolute Percentage Error:     {mape:.2f}%")
    print(f"   Maximum Error:                      ${max_error:.2f}")
    print(f"   Minimum Error:                      ${min_error:.2f}")
    print(f"   Standard Deviation of Errors:       ${std_error:.2f}")
    
    print("\n2. ACCURACY METRICS")
    print("-" * 70)
    print(f"   R² Score (Coefficient of Determination): {r2:.4f}")
    print(f"   Variance Explained:                    {r2*100:.2f}%")
    print(f"   Percentage Accuracy:                  {percentage_accuracy:.2f}%")
    print(f"   Directional Accuracy:                  {directional_accuracy:.2f}%")
    
    print("\n3. BIAS ANALYSIS")
    print("-" * 70)
    print(f"   Mean Percentage Error (Bias):        {mean_percentage_error:.2f}%")
    if mean_percentage_error > 0:
        print("   Model tends to:                        OVERESTIMATE prices")
    elif mean_percentage_error < 0:
        print("   Model tends to:                        UNDERESTIMATE prices")
    else:
        print("   Model shows:                           NO SIGNIFICANT BIAS")
    
    print("\n4. CONTEXT METRICS")
    print("-" * 70)
    print(f"   Average Stock Price (Test Set):       ${avg_price:.2f}")
    print(f"   RMSE as % of Average Price:           {(rmse/avg_price)*100:.2f}%")
    print(f"   MAE as % of Average Price:             {(mae/avg_price)*100:.2f}%")
    
    print("\n" + "="*70)
else:
    print("Variables not available. Please run implementation.ipynb first.")